In [1]:
import os

print(os.getcwd())
os.chdir('C:\\Users\\lee\\practice problems\\loan prediction')
print(os.getcwd())

C:\Users\lee
C:\Users\lee\practice problems\loan prediction


In [2]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train = train.drop('Loan_ID', axis=1)

In [14]:
def describe_data(df):
    
    print('Data types:')
    print(df.dtypes)

    print()
    
    print('Rows and columns:')
    print(df.shape)
    
    print()
    
    print('Column names:')
    print(df.columns)
    
    print()
    
    print('Null values:')
    print(df.apply(lambda x: sum(x.isnull()) / len(df)))

describe_data(train)

Data types:
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

Rows and columns:
(614, 12)

Column names:
Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

Null values:
Gender               0.021173
Married              0.004886
Dependents           0.024430
Education            0.000000
Self_Employed        0.052117
ApplicantIncome      0.000000
CoapplicantIncome    0.000000
LoanAmount           0.035831
Loan_Amount_Term     0.022801
Credit_History       0.081433
Property_Area        0.000000
Loan_

In [17]:
numeric_features = train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = train.select_dtypes(include=['object']).drop(['Loan_Status'], axis=1).columns

In [22]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [23]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

In [24]:
from sklearn.model_selection import train_test_split
X = train.drop('Loan_Status', axis=1)
y = train['Loan_Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [27]:
from sklearn.linear_model import LogisticRegression

lr = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

In [28]:
lr.fit(X_train, y_train)
print("Model score: %.3f" % lr.score(X_test, y_test))

Model score: 0.805


In [29]:
test_no_id = test.drop("Loan_ID", axis=1)

In [30]:
test_predictions = lr.predict(test_no_id)

In [35]:
Loan_ID = test['Loan_ID']
submission_df_1 = pd.DataFrame({
    "Loan_ID": Loan_ID,
    "Loan_Status": test_predictions
})

In [37]:
submission_df_1.to_csv('submission_1.csv', index=False)